# Reclamações de Clientes Bancários - Conjunto de dados


Este conjunto de dados captura um instantâneo das reclamações de clientes feitas a um banco fictício, permitindo-nos aprofundar as diversas questões e preocupações levantadas pelos seus clientes.

Estas reclamações abrangem uma vasta gama de tópicos relacionados com serviços e interações bancárias, desde problemas com contas a disputas de cartão de crédito e preocupações com roubo de identidade.

## Características Principais Do Conjunto de dados (ou Dataframe)

### Características principais

**Informações do cliente**: o conjunto de dados inclui detalhes sobre os clientes que apresentaram as reclamações, como IDs de clientes, nomes e endereços de e-mail. Essas informações fornecem contexto sobre os indivíduos que tiveram problemas com o banco.

**Tipo de reclamação**: Cada reclamação foi categorizada em um dos vários tipos predefinidos. Essas categorias abrangem preocupações comuns relacionadas ao setor bancário, como "Problema de conta", "Disputa de cartão de crédito", "Problemas de empréstimo", "Roubo de identidade" e uma categoria geral rotulada como "Outros".

**Descrição da reclamação**: o cerne do conjunto de dados está nas descrições detalhadas de cada reclamação. Os clientes expressaram suas queixas, descreveram os problemas que encontraram e forneceram contexto para suas reclamações por meio dessas descrições textuais. Estas narrativas contêm informações valiosas sobre a natureza e a gravidade dos problemas.

**Contexto Adicional**: Para melhorar a compreensão de cada reclamação, foram incluídas informações complementares. Essas informações consistem na data em que a reclamação foi apresentada, a localização da agência bancária onde a reclamação foi recebida e o nome do bancário responsável pelo tratamento da reclamação.

## Codigo Python e as Analises

### Bibliotecas Utilizadas

In [ ]:
pip install WordCloud
pip install TextBlob
pip install scikit-learn

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

### Carregando o Conjunto de dados

In [3]:
df = pd.read_csv('improved_bank_complaints_dataset.csv', delimiter=',');df.head()

,Customer_ID,Customer_Name,Customer_Email,Date_of_Complaint,Complaint_Type,Complaint_Description,Branch_Location,Employee_Name
0,87999924,Denise Saunders,sheajessica@example.net,2022-04-19T05:43:28,Identity Theft,Parent democratic security crime. Big degree n...,Yatesland,David Smith
1,62126745,Michael Wells,antonio68@example.com,2022-01-28T20:12:02,Account Issue,Within our worker key father coach. When up re...,Tracyborough,Ernest Anderson
2,61875880,Peter Gonzales,danielmark@example.org,2021-11-27T15:24:59,Other,Various community physical next growth item. C...,West Michaelfort,Amanda Parker
3,93494060,Joseph Castro,greenrichard@example.org,2022-09-27T09:17:06,Other,Onto on share. Season turn size spring. She th...,Bartonchester,Colleen Herman
4,39811773,Barbara Burch,bjones@example.com,2022-06-22T17:36:51,Other,Environment letter remain member clear just co...,Marymouth,Joshua Sloan


### Estrutura da base de dados

In [4]:
num_rows, num_columns = df.shape

print(f"Numero de linhas: {num_rows}")
print(f"Numero de colunas: {num_columns}")

Numero de linhas: 500000
Numero de colunas: 8


### Variaveis do Conjunto de dados e o Contexto

In [5]:
variable_names = df.columns.tolist()

print("Variable Names:")
for variable in variable_names:
    print(variable)

Variable Names:
Customer_ID
Customer_Name
Customer_Email
Date_of_Complaint
Complaint_Type
Complaint_Description
Branch_Location
Employee_Name


**Customer_ID**: Esta variável representa um número de identificação único atribuído a cada cliente que apresentou uma reclamação. Serve como uma forma de distinguir clientes individuais e pode ser usado para rastreamento e referência.

**Customer_Name**: Esta variável contém o nome completo do cliente que fez a reclamação. Fornece uma referência à identidade do cliente e permite a personalização na abordagem das suas preocupações.

**Customer_Email**: esta variável contém o endereço de e-mail associado ao cliente. Os endereços de e-mail são comumente usados ​​para fins de comunicação e contato, tornando mais fácil para o banco entrar em contato com os clientes sobre suas reclamações ou problemas.

**Date_of_Complaint**: Esta variável registra a data e hora em que a reclamação foi apresentada. Ele oferece um carimbo de data/hora em que o cliente expressou sua preocupação, que pode ser usado para rastrear tempos de resposta e tendências no envio de reclamações.

**Complaint_Type**: Esta variável categoriza cada reclamação em tipos ou categorias específicas. Ajuda a classificar a natureza do problema, quer se trate de problemas relacionados com contas, disputas de cartões de crédito, problemas de empréstimos, preocupações com roubo de identidade ou se caia numa categoria geral “Outros”.

**Complaint_Description**: A variável Complaint_Description contém a narrativa detalhada fornecida pelo cliente para descrever o problema ou preocupação. Este texto fornece contexto e detalhes sobre a reclamação, permitindo uma compreensão mais profunda do problema.

**Branch_Location**: Esta variável indica a localização da agência bancária onde a reclamação foi recebida ou tratada. Ajuda a identificar a agência física envolvida na reclamação, o que pode ser útil para rastrear problemas ou desempenho específicos da agência.

**Employee_Name**: Employee_Name contém o nome do funcionário do banco responsável por tratar ou resolver a reclamação do cliente. Associa a reclamação ao colaborador que prestou assistência ou apoio durante o processo de resolução.

## Verificando se o conjunto de dados apresenta valores duplicados ou ausentes

In [6]:
import pandas as pd

# Check for duplicate rows
duplicate_rows = df[df.duplicated()]

# Check for missing values
missing_values = df.isnull().sum()

if not duplicate_rows.empty:
    print("Linhas duplicadas:")
    print(duplicate_rows)
else:
    print("Nenhuma linha duplicada encontrada")

if missing_values.any():
    print("\nMissing Values:")
    print(missing_values)
else:
    print("\n Não Valores ausentes encontrados")


Nenhuma linha duplicada encontrada

 Não Valores ausentes encontrados


## Criando um modelo de Classificação - Multinomial Naive Bayes

Multinomial Naive Bayes é um algoritmo de classificação popular comumente usado em processamento de linguagem natural (PNL) e tarefas de classificação de texto. Baseia-se nos princípios do teorema de Bayes e é particularmente adequado para trabalhar com dados discretos, como documentos de texto. Este algoritmo é caracterizado pela sua simplicidade e eficiência.

No Multinomial Naive Bayes, a ideia fundamental é modelar a distribuição de probabilidade de características (palavras ou termos em dados de texto) dentro de cada classe ou categoria. Assume que os recursos são condicionalmente independentes, o que significa que a presença ou ausência de uma palavra em um documento não afeta a presença ou ausência de outras palavras. Embora esta seja uma simplificação que nem sempre é verdadeira em dados de texto do mundo real, muitas vezes funciona surpreendentemente bem na prática.

Para classificar um novo documento, o Multinomial Naive Bayes calcula a probabilidade desse documento pertencer a cada classe com base nas frequências observadas de palavras ou recursos nos dados de treinamento. Em seguida, atribui a classe com a maior probabilidade como a classe prevista para o documento. Multinomial Naive Bayes é particularmente eficaz para tarefas de classificação de texto, como detecção de spam de e-mail ou análise de sentimento, e é conhecido por sua velocidade e baixos requisitos computacionais. No entanto, seu desempenho pode ser limitado ao lidar com relacionamentos complexos entre recursos ou quando a suposição de independência de recursos é violada.

Em resumo, Multinomial Naive Bayes é um algoritmo de classificação simples, mas poderoso, amplamente utilizado em PNL e classificação de texto. Ele aproveita o teorema de Bayes e a suposição de independência de recursos para fazer previsões sobre a classe de um novo documento com base nas frequências de palavras observadas nos dados de treinamento. Embora possa não capturar relações complexas entre palavras, é eficiente, fácil de implementar e muitas vezes oferece resultados competitivos em diversas aplicações relacionadas a texto.

In [8]:
# Criando um mapeamento de tipos de reclamação para rótulos numéricos
complaint_type_mapping = {
    'Account Issue': 0,
    'Credit Card Dispute': 1,
    'Loan Problems': 2,
    'Identity Theft': 3,
    'Other': 4
}

# Criando uma nova coluna com rótulos numéricos
df['Complaint_Type_Label'] = df['Complaint_Type'].map(complaint_type_mapping)

# Divida o conjunto de dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(
    df['Complaint_Description'],
    df['Complaint_Type_Label'],
    test_size=0.2,
    random_state=42
)

# Crie vetorizadores TF-IDF para recursos de texto
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # You can adjust max_features as needed

# Transforme os dados de treinamento e teste
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Inicialize e treine um classificador de texto (por exemplo, Multinomial Naive Bayes)
classifier = MultinomialNB()
classifier.fit(X_train_tfidf, y_train)

# Faça previsões no conjunto de testes
y_pred = classifier.predict(X_test_tfidf)



                     precision    recall  f1-score   support

      Account Issue       0.20      0.26      0.22     20168
Credit Card Dispute       0.20      0.18      0.19     19641
      Loan Problems       0.20      0.23      0.22     20121
     Identity Theft       0.20      0.16      0.18     19861
              Other       0.21      0.18      0.19     20209

           accuracy                           0.20    100000
          macro avg       0.20      0.20      0.20    100000
       weighted avg       0.20      0.20      0.20    100000



O código acima realiza as seguintes atividades:

**(1)** Mapeia os tipos de reclamação em rótulos numéricos para classificação.

**(2)** Divide o conjunto de dados em conjuntos de treinamento e teste.

**(3)** Usa vetorização TF-IDF para converter dados de texto em recursos numéricos.

**(4)** Inicializa e treina um classificador Multinomial Naive Bayes.

**(5)** Faz previsões no conjunto de testes e imprime um relatório de classificação com métricas como precisão, recall e pontuação F1 para cada tipo de reclamação.

In [9]:
# Imprimir relatório de classificação
print(classification_report(y_test, y_pred, target_names=complaint_type_mapping.keys()))

                     precision    recall  f1-score   support

      Account Issue       0.20      0.26      0.22     20168
Credit Card Dispute       0.20      0.18      0.19     19641
      Loan Problems       0.20      0.23      0.22     20121
     Identity Theft       0.20      0.16      0.18     19861
              Other       0.21      0.18      0.19     20209

           accuracy                           0.20    100000
          macro avg       0.20      0.20      0.20    100000
       weighted avg       0.20      0.20      0.20    100000



**Precision** : A precisão mede a precisão das previsões positivas. Neste contexto, informa a percentagem de reclamações corretamente classificadas para cada categoria. Por exemplo, para “Problema de Conta”, apenas 20% das reclamações previstas foram corretamente classificadas como tal.

**Recall** : Recall mede a capacidade do classificador de identificar corretamente todas as instâncias relevantes. Neste contexto, informa qual a percentagem de reclamações reais de cada categoria que foram classificadas corretamente. Por exemplo, para “Problema de conta”, apenas 26% das reclamações reais foram classificadas corretamente.

**f1-score** : A pontuação F1 é a média harmônica de precisão e recuperação. Ele fornece um equilíbrio entre precisão e recuperação. Neste contexto, indica o equilíbrio entre classificar corretamente as reclamações e identificar corretamente todas as reclamações reais. Por exemplo, a pontuação F1 para “Problema de conta” é 0,22.

**support** : O suporte indica o número de instâncias reais para cada categoria no conjunto de teste.

**Precisão** : A precisão geral do modelo é de 20%. Isto significa que, em média, apenas 20% das reclamações são corretamente classificadas nas respetivas categorias.

O desempenho da classificação parece ser bastante baixo, com valores de precisão, recall e pontuação F1 em torno de 0,20. Isto sugere que o atual modelo de classificação de texto pode não distinguir eficazmente entre as diferentes categorias de reclamações. Para tentarmos melhorar esses resultados vamos aplicar outras tecnicas.


Neste código abaixo, iplementaremos um pipeline de classificação de texto com vetorização TF-IDF e um classificador Multinomial Naive Bayes. O código também demonstra o uso de validação cruzada k-fold para estimar melhor o desempenho do modelo.

In [15]:
# Divida o conjunto de dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(
    df['Complaint_Description'],
    df['Complaint_Type'],
    test_size=0.2,
    random_state=42
)

# Criando um pipeline de classificação de texto com vetorização TF-IDF e um classificador
text_clf = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000, ngram_range=(1, 2))),  
    ('clf', MultinomialNB())
])

# Treinando o modelo com os dados de treinamento
text_clf.fit(X_train, y_train)

# Avaliando o modelo no conjunto de teste
y_pred = text_clf.predict(X_test)

# Print relatório de classificação
print(classification_report(y_test, y_pred))

# Execute a validação cruzada k-fold para avaliar o desempenho do modelo
scores = cross_val_score(text_clf, df['Complaint_Description'], df['Complaint_Type'], cv=5)
print(f"Precisão média de validação cruzada: {scores.mean()}")

# Optional: Fine-tune hyperparameters, try different classifiers, or implement other improvements as needed.


                     precision    recall  f1-score   support

      Account Issue       0.20      0.26      0.23     20168
Credit Card Dispute       0.20      0.18      0.19     19641
     Identity Theft       0.20      0.16      0.18     19861
      Loan Problems       0.20      0.23      0.21     20121
              Other       0.21      0.18      0.19     20209

           accuracy                           0.20    100000
          macro avg       0.20      0.20      0.20    100000
       weighted avg       0.20      0.20      0.20    100000

Precisão média de validação cruzada: 0.200562


Apesar da aplicação de técnicas como Naive Bayes e da criação de um pipeline, o desempenho da classificação não mostrou melhoria significativa neste conjunto de dados. É essencial reconhecer que a classificação de texto pode ser uma tarefa desafiadora e que melhorar o desempenho muitas vezes requer uma combinação de abordagens.

## Conclusão

Como o intuito desse relatorio é aplicar um tecnica de classificação , verificamos que mesmo utilizando um modelo (ou tema) avançãdo como naives bayes , nosso conjunto de dados não obteve bons desempenhos com o modelo utilizado , mesmo quando criado um pipeline e realizando alguns ajustes.

Como sugestão indicaria alguns outras formas de tentar resolver esse problema que encontramos em classificar nosso conjunto de dados.

(1) **Aumento de dados** :

Aumentar o conjunto de dados criando variações de reclamações existentes. Técnicas como parafrasear, adicionar sinônimos ou alterar a estrutura das frases podem ajudar a gerar exemplos de treinamento mais diversos.

(2) **Modelos de aprendizagem profunda (ou Deep Learning)** :

Explore modelos baseados em aprendizagem profunda, como redes neurais convolucionais (CNNs), redes neurais recorrentes (RNNs) ou arquiteturas baseadas em transformadores (por exemplo, BERT, RoBERTa) projetadas especificamente para tarefas de processamento de linguagem natural.
